RT-DETR --> ONNX

In [1]:
import torch
import torch.nn as nn
import torchvision
import onnx
import onnxruntime as ort
import numpy as np
from rtdetrv2_pytorch.src.core import YAMLConfig

# **路径配置**
config_path = "/home/guoy/led_detection/training/RT-DETR/rtdetrv2_pytorch/configs/rtdetrv2/rtdetrv2_r50vd_m_7x_coco.yml"
pth_path = "/home/guoy/led_detection/training/RT-DETR/output/rtdetrv2_r50vd_m_7x_coco_final/best.pth"
onnx_path = "/home/guoy/led_detection/training/RT-DETR/model_v1.onnx"

# **加载 PTH 模型**
cfg = YAMLConfig(config_path, resume=pth_path)
checkpoint = torch.load(pth_path, map_location="cpu")
state_dict = checkpoint.get("ema", checkpoint.get("model"))["module"]
cfg.model.load_state_dict(state_dict)

class ONNXModel(nn.Module):
    """ONNX 兼容的 RT-DETR，包含后处理"""
    def __init__(self, model, postprocessor):
        super().__init__()
        self.model = model.deploy()
        self.postprocessor = postprocessor.deploy()
        
    def forward(self, images):
        outputs = self.model(images)
        logits, boxes = outputs["pred_logits"], outputs["pred_boxes"]

        # **🔥 确保置信度正确**
        scores = torch.sigmoid(logits)

        # **🔥 确保坐标转换正确**
        boxes = torchvision.ops.box_convert(boxes, in_fmt='cxcywh', out_fmt='xyxy')

        # **🔥 获取类别标签**
        labels = torch.argmax(logits, dim=-1)  # 获取最高分的类别索引

        return scores, boxes, labels

# **构建 ONNX 兼容模型**
onnx_model = ONNXModel(cfg.model, cfg.postprocessor)
onnx_model.eval()

# **导出 ONNX**
dummy_input = torch.randn(1, 3, 640, 640)

torch.onnx.export(
    onnx_model,
    (dummy_input,),
    onnx_path,
    input_names=['images'],
    output_names=['scores', 'boxes', 'labels'],
    opset_version=16,
    dynamic_axes={'images': {0: 'batch_size'}},
    do_constant_folding=True,
)
print(f"✅ ONNX 模型已成功导出至: {onnx_path}")


/tmp/ipykernel_359481/95100617.py:16: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(pth_path, map_location="cpu")


Load PResNet50 state_dict


/home/guoy/led_detection/training/RT-DETR/rtdetrv2_pytorch/src/zoo/rtdetr/rtdetrv2_decoder.py:141: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if reference_points.shape[-1] == 2:
/home/guoy/led_detection/training/RT-DETR/rtdetrv2_pytorch/src/zoo/rtdetr/rtdetrv2_decoder.py:145: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  elif reference_points.shape[-1] == 4:


✅ ONNX 模型已成功导出至: /home/guoy/led_detection/training/RT-DETR/model_v1.onnx
